In [1]:
# import
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Read API
API_KEY = ''

with open('ALPHAVANTAGE_API_KEY') as f:
    API_KEY = f.read()

ts = TimeSeries(API_KEY, output_format='pandas', indexing_type='integer') 
ti = TechIndicators(key=API_KEY, output_format='pandas')

In [51]:
df, dm = ts.get_daily('META')

In [52]:
df.head()

,index,1. open,2. high,3. low,4. close,5. volume
index,,,,,,
0,2024-08-02,489.00,501.1450,476.15,488.14,24044658.0
1,2024-08-01,521.00,527.1709,492.10,497.74,43083082.0
2,2024-07-31,471.02,476.5000,466.75,474.83,24285777.0
3,2024-07-30,467.00,472.7300,456.70,463.19,11390439.0
4,2024-07-29,469.88,473.9600,465.02,465.71,11339630.0


In [53]:
df['day'] = pd.to_datetime(df['index']).dt.day_of_week

In [54]:
df['PnL'] = np.where((df['4. close']-df['1. open'])<0, -1, 1)
df['PnL']

index
0    -1
1    -1
2     1
3    -1
4    -1
     ..
95    1
96   -1
97   -1
98    1
99    1
Name: PnL, Length: 100, dtype: int64

In [49]:
# https://medium.com/@smitpate08/unleashing-the-power-of-categorical-relationships-with-pandas-in-python-82fbe3917338
# based on this doc, relationship testing
from scipy.stats import chi2_contingency

def check_relationship(data, alpha = 0.05):
    # Get all categorical columns in the DataFrame
    categorical_columns = data.columns
    results = []
    
    # Create a copy of the dataframe to avoid modifying the original
    data_copy = data.copy()
    
    # Remove rows containing missing values
    data_copy = data_copy.dropna(subset=categorical_columns)
    
    # Iterate over each pair of categorical columns
    for i in range(len(categorical_columns)):
        for j in range(i+1, len(categorical_columns)):
            col1 = categorical_columns[i]
            col2 = categorical_columns[j]
            
            # Create the contingency table for the two columns
            contingency_table = pd.crosstab(data_copy[col1], data_copy[col2])
            
            # Calculate the chi-squared test of independence
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            
            # Check if we reject the null hypothesis (i.e., if the columns are dependent)
            reject_null_hypothesis = p < alpha
            dependency = reject_null_hypothesis
            
            # Add the results to the list of results
            results.append({'column': col1, 'column_compared_with': col2, 'p-value': p, 'Null hypothesis rejected': reject_null_hypothesis, 'dependency': dependency})
    return pd.DataFrame(results).sort_values(by='p-value', ascending=False)

In [55]:
check_relationship(df[['day', 'PnL']])

,column,column_compared_with,p-value,Null hypothesis rejected,dependency
0,day,PnL,0.438971,False,False


In [61]:
df, dm = ts.get_weekly('META')

In [62]:
df['day'] = pd.to_datetime(df['index']).dt.weekofyear
df['PnL'] = np.where((df['4. close']-df['1. open'])<0, -1, 1)

/var/folders/ql/t_pwhj2x01x9pf367dwbv_5c0000gn/T/ipykernel_12478/3216668950.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['day'] = pd.to_datetime(df['index']).dt.weekofyear


In [63]:
check_relationship(df[['day', 'PnL']])

,column,column_compared_with,p-value,Null hypothesis rejected,dependency
0,day,PnL,0.766047,False,False
